## AWS credentials 

In [ ]:
import os
import boto3
from dotenv import load_dotenv

# === Load environment variables ===
load_dotenv()

bucket = os.getenv("AWS_BUCKET")
prefix = os.getenv("AWS_PREFIX", "llm_pipeline")  # default value

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Optional: initialize boto3 client safely
s3_client = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

print(f"✅ Connected to S3 bucket: {bucket}/{prefix}")


## Csv to jsonL

In [20]:
import pandas as pd
import json

input_csv = 'input/generated_headline-to_finbert.csv'

output_jsonl = 'headline.jsonl'

df = pd.read_csv(input_csv)
df['id'] = df['id'].astype(str)
print(df.columns)

with open(output_jsonl, 'w', encoding='utf-8') as f:
    for record in df.to_dict(orient='records'):
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

Index(['id', 'headline', 'symbol', 'name', 'content', 'generated_headline'], dtype='object')


In [21]:
import tarfile
import os

filepath ='headline.jsonl'
filename = 'headline'
with tarfile.open(f'{filename}.tar.gz', "w:gz") as tar:
    tar.add(filepath, arcname=f'{filename}.jsonl')

## Upload code files to S3

In [10]:
import boto3

s3 = boto3.client('s3', aws_access_key_id=api_key_aws, aws_secret_access_key=api_key_secret_aws)

#s3.upload_file('process.py', bucket, f'{prefix}/code/process.py')
#s3.upload_file('llm_utils.py', bucket, f'{prefix}/code/llm_utils.py')
#s3.upload_file('requirements.txt', bucket, f'{prefix}/code/requirements.txt')
s3.upload_file('code.tar.gz', bucket, f'{prefix}/code/code.tar.gz')


## directory to tar.gz

In [9]:
import tarfile
import os

with tarfile.open('code.tar.gz', "w:gz") as tar:
    for filename in os.listdir('code'):
        filepath = os.path.join('code', filename)
        tar.add(filepath, arcname=filename)

In [2]:
from transformers import AutoModelForCausalLM
help(AutoModelForCausalLM.from_pretrained)

Help on method from_pretrained in module transformers.models.auto.auto_factory:

from_pretrained(*model_args, **kwargs) class method of transformers.models.auto.modeling_auto.AutoModelForCausalLM
    Instantiate one of the model classes of the library (with a causal language modeling head) from a pretrained model.
    
    The model class to instantiate is selected based on the `model_type` property of the config object (either
    passed as an argument or loaded from `pretrained_model_name_or_path` if possible), or when it's missing, by
    falling back to using pattern matching on `pretrained_model_name_or_path`:
    
        - **bart** -- [`BartForCausalLM`] (BART model)
        - **bert** -- [`BertLMHeadModel`] (BERT model)
        - **bert-generation** -- [`BertGenerationDecoder`] (Bert Generation model)
        - **big_bird** -- [`BigBirdForCausalLM`] (BigBird model)
        - **bigbird_pegasus** -- [`BigBirdPegasusForCausalLM`] (BigBird-Pegasus model)
        - **biogpt** -- [`B